---

![](Images/SS27.png)

---

In [1]:
# Setting up a custom stylesheet in IJulia
# New in 0.6
file = open("style.css") # A .css file in the same folder as this notebook file
styl = readstring(file) # Read the file
HTML("$styl") # Output as HTML

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.CodeMirror{\r\n        font-family: \"PT Mono\";\r\n        font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

In [ ]:
for p in ("BenchmarkTools","PyCall", "PyPlot")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using BenchmarkTools, PyCall, PyPlot

---

![](Images/SS01.png)

---

# Getting started with Julia

* JuliaBox&nbsp;&nbsp;:- [http://juliabox.com](http://juliabox.com)
* JuliaPro&nbsp;&nbsp;&nbsp;:- [http://juliacomputing.com](http://juliacomputing.com)
* JuliaLang&nbsp;:- [http://julialang.org](http://juliacomputing.org)

---


## Julia code is *just what you would expect* 

In [ ]:
# Functions can be defined in a single line or as a block
# They can restrict the types passed 

f1(x::Real) = sin(x.^2)./x;

# Should be f1(x::Real) = (abs(x) < 10.0E-7) ? 0.0 : sin(x.^2)./x;
# To allow for x == 0.0

f1(1.3)

In [ ]:
f1(1.3 + 2.2im)  # This will FAIL as we have defined the function above

In [ ]:
# This one works with compelx numbers too

f2(x::Number) = sin(x.^2)./x;

f2(1.3 + 2.2im)

In [ ]:
# We can use f() to plot a graph

using PyPlot
t = -2π:0.02:2π; 
y = [f1(u) for u in t];

plot(t, y)

![](Images/SS10a.png)

### Dispatch: A simple function

In [ ]:
incr(x) = x + 1;

In [ ]:
incr(1)

In [ ]:
incr(2.1)

In [ ]:
incr(5//7)

In [ ]:
@code_native incr(1)

In [ ]:
@code_native incr(2.1)

In [ ]:
@code_native incr(5//7)

In [ ]:
@code_typed incr(2.1)

---

### More on Multiple Dispatch


In [ ]:
# Julia can use alternative characters.

(€)(x::T,c::T) where T<:Number = x^2 + c

In [ ]:
€(2.7,3.8)

In [ ]:
€(2//7,3//8)

In [ ]:
z = [€(Complex(rand(),rand()),Complex(rand(),rand())) for i in 1:25]
zz = reshape(z,5,5)

In [ ]:
det(zz)

In [ ]:
ww = zz.*conj(zz)

In [ ]:
norm.(zz)   # 'dot' is a shortcut for the map function:  map(z -> norm(z), zz)

### Mandelbrot set

In [ ]:
function mandel(z::Complex, N::Integer)
  c = z
  for n = 1:N
    (norm(z) > 2) && return false
    z = €(z, c)
  end
  return true
end

In [ ]:
mandel(0.1+0.1*im, 30)

In [ ]:
m0 = [mandel(u + v*im, 30) for u in -1.5:0.005:0.5, v in -1.0:0.005:1.0];
(mx,my) = size(m0)

In [ ]:
m1 = Array{Int32,2}(mx,my)
for i in 1:mx, j in 1:my
  m1[i,j] = m0[i,j] ? 0 : 255
end

In [ ]:
function create_pgmfile(img, outf::String)
    s = open(outf, "w")
    write(s, "P5\n")    
    n, m = size(img)
    write(s, "$m $n 255\n")
    for i=1:n, j=1:m
      write(s, UInt8(img[j,i]))
    end
    close(s)
end

create_pgmfile(m1,"./mandelbrot.pgm")

In [ ]:
run(`display ./mandelbrot.pgm`)  # This executes an external task

---

### Let's investigate a function to evaluate PI

![](Images/eval-pi.png)

In [ ]:
function mypi(n::Integer)
  @assert n > 0
  k = 0
  for i = 1:n
    if (rand()^2 + rand()^2 <= 1)
       k +=1
    end
  end
  4.0 * (k / n)
end

In [ ]:
macroexpand(:(@assert n > 0))

In [ ]:
N = 10_000_000;
pi2 = mypi(N)

In [ ]:
@printf "My value for PI is %.5f\n" pi2

In [ ]:
# Look at how the work-horse line is parsed

dump(:(rand()^2 + rand()^2 < 1))

In [ ]:
# In a more compact form

Meta.show_sexpr(:(rand()^2 + rand()^2 < 1))

In [ ]:
# ... and the lowered code

@code_lowered mypi(10000000)

In [ ]:
# ... and the typed code

@code_typed mypi(10000000)

In [ ]:
# ... and the native (x86) CODE

@code_native mypi(10000000)

#### Timing the function

In [ ]:
# Using a macro

@time mypi(N)

In [ ]:
macroexpand(:(@time pi2 = mypi(N)))

In [ ]:
# Alternatively with teh BenchmarkTools package

@benchmark mypi(N) samples=10

---

## Number type hierarchy in Julia

![](Images/type-hierarchy-numbers.png)

---

### Define a "Natural" *( i.e. non-negative integer )*

In [ ]:
# Need a simple constructor to check n >= 0

struct Natural
  n::Integer
  function Natural(n::Integer)
    n < zero(Integer) && throw(ArgumentError("Invalid Natural: not non-negative"))
    new(n)
  end
end

n0 = Natural(3)

In [ ]:
# With just a type definition we can NOT do any arithmetic operations
# ... unless we define how to perform them

n1 = Natural(2)
n2 = Natural(5)

n1 + n2

In [ ]:
# However this is not to difficult

import Base.+
+(n1::Natural, n2::Natural) = Natural(n1.n + n2.n)

In [ ]:
n1 + n2

In [ ]:
# This is an ugly way to display the value of the natural
# but we can also redefine how this is presented

import Base.show

function show(io::IO, k::Natural)
    show(io, k.n)
end

In [ ]:
n1 + n2

### A Simple Type in 4-D space

In [ ]:
struct Vec4D
  w::Float64
  x::Float64
  y::Float64
  z::Float64
end

In [ ]:
import Base: +, -, *, /
import Base: ==, <, >
import Base: zero, one

In [ ]:
zero(::Type{Vec4D}) = Vec4D(zero(Float64), zero(Float64), zero(Float64), zero(Float64))
one(::Type{Vec4D}) = Vec4D(one(Float64), one(Float64), one(Float64), one(Float64))

In [ ]:
methods(one)

In [ ]:
? one

In [ ]:
+(u::Vec4D, v::Vec4D) = Vec4D(u.w + v.w, u.x + v.x, u.y + v.y, u.z + v.z)

-(u::Vec4D) = Vec4D(-u.w, -u.x, -u.y, -u.z)
-(u::Vec4D, v::Vec4D) = u + (-v)

*(u::Vec4D, v::Vec4D)   = Vec4D(u.w*v.w, u.x*v.x, u.y*v.y, u*z*v.z)

*(a::Float64, u::Vec4D) = Vec4D(a*u.w, a*u.x, a*u.y, a*u.z)
*(u::Vec4D, a::Float64) = a*u

/(u::Vec4D, a::Float64) = (1.0/a)*u;

In [ ]:
import Base:norm

In [ ]:
dist(u::Vec4D, v::Vec4D) = ((u.w - v.w)^2 + (u.x - v.x)^2 + (u.y - v.y)^2 + (u.z - v.z)^2)^0.5
norm(u::Vec4D) = (u.w^2 + u.x^2 + u.y^2 + u.z^2)^0.5

# We could also define norm as:  norm(u::Vec4D) = dist(u, zero(Vec4D))

convert(::Type{Vec4D}, a::Real) = Vec4D(a,zero(a))
promote_rule(::Type{Vec4D}, ::Type{<:Number}) = Vec4D;

In [ ]:
p = Vec4D(1.2,3.2,2.7,3.9)

In [ ]:
u = Vec4D(rand(), rand(), rand(), rand())

In [ ]:
norm(u)

In [ ]:
vv = [Vec4D(rand(), rand(), rand(), rand()) for i = 1:1000000];

In [ ]:
# Volume of the unit 4-ball is 2*π*π
# The count is 1/16th of the volume
# see: https://en.wikipedia.org/wiki/Volume_of_an_n-ball

k = 0
for i in 1:length(vv)
  if norm(vv[i]) < 1.0 k +=1 end
end

@printf "Estimate of PI is %9.5f\n" sqrt(32.0*k/length(vv))

In [ ]:
# Rather than specifying the type of component as Float64 
# We can use a general parameter {T}
# Constraining this to be an number

struct Vec4T{T<:Number} 
  w::T
  x::T
  y::T
  z::T
end

In [ ]:
# More generally we can pass the number of components {N}
# The operations now will need to be in terms of arrays

 struct VecN{T, N}
   x::Array{T,N}
end

In [ ]:
vv = VecN([1.2, 3.7, 5.1, 4.0, 2.2])

In [ ]:
fieldnames(vv)

In [ ]:
vv.x[2]   # Recall that like many scientific programming languages, Julia is 1-based


---


## A _(little)_ bit on macros

In [ ]:
using BenchmarkTools

### Horner's Method

<h4>y = a + bx + cx<sup>2</sup> +dx<sup>3</sup> + e<sup>4</sup></h4>

In [ ]:
# Evaluate a polynomial with a function
# This is NOT a macro

function poly_native(x, a...)
  p=zero(x)
  for i = 1:length(a)
    p = p + a[i] *  x^(i-1)
  end
  return p
end

f_native(x) = poly_native(x,1,2,3,4,5)
f_native(2.1)

In [ ]:
t = @benchmark f_native(2.1)
fieldnames(t)

In [ ]:
median(t)

In [ ]:
# Horners method
# Neither IS this!

function poly_horner(x, a...)
  b=zero(x)
  for i = length(a):-1:1
    b = a[i] + b * x
  end
  return b
end

# f -> (((5*x + 4)*x + 3)*x + 2)*x + 1
#
f_horner(x) = poly_horner(x,1,2,3,4,5)
f_horner(2.1)

In [ ]:
t = @benchmark f_horner(2.1)
median(t)

In [ ]:
# Define mad(x,a,b)
# [Julia has this function too :- muladd(x,a,b)]

mad(x,a,b) = a*x + b
mad(2.1,5,4)

In [ ]:
# And NOW use horner's method in a macro
# [ p... is a variable list of arguments passed as an array]

macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(mad(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

In [ ]:
@horner 2.1 1 2 3 4 5   # => This can also be invoked as: @horner(2.1, 1, 2, 3, 4, 5)

In [ ]:
macroexpand(:(@horner 2.1 1 2 3 4 5))

In [ ]:
t = @benchmark @horner 2.1 1 2 3 4 5
median(t)

---

## Interoperability

- #### Julia can connect to Python *(PyCall)*, R *(RCall)*, Java *(JavaCall)* etc.
- #### C++ is harder but now possible
- #### The basis of all of these is the built-in C interface *(ccall)*
- #### Because of this Julia has no need of a formal API
- #### If a interop-package has NOT yet been written, it is possible to use the O/S
- #### Julia can run *pipes* and capture *standard output*


In [ ]:
# Simple call to a C-routine
# Can be used to set the random number seed

systime()   = ccall((:clock,"libc"),Int32,())
randomize() = srand(systime())

In [ ]:
systime()

In [ ]:
#= Link to a C routine

#include<math.h>

double horner(double x, double aa[], long n) {
  long i;
  double s = aa[n-1];
  if (n > 1) { 
    for (i = n-2; i >= 0; i--) {
      s = s*x + aa[i];
    }
  }
  return s;
}

// Build a dynamic library (on OSX) as:

// clang -c horner.c
// libtool -dynamic horner.o -o libmyfuns.dylib  -lSystem -macosx_version_min 10.13
// sudo cp libmyfuns.dylib /usr/local/lib

=#

run(`nm /usr/local/lib/libmyfuns.dylib`)

In [ ]:
x = 2.1;
aa = [1.0, 2.0, 3.0, 4.0, 5.0];

ccall((:horner,"libmyfuns.dylib"),Cdouble,(Cdouble,Ptr{Cdouble},Clong),x,aa,length(aa))

---

### Return to computing PI

In [ ]:
# Compute PI in 'C"

C_code = """
#include <stddef.h>
#include <stdlib.h>

double c_pi(long n) {
    long k = 0L;
    float rmax = (float) RAND_MAX;

    for (long i = 0L; i < n; i++) {
        float x = ((float) rand())/rmax;
        float y = ((float) rand())/rmax;
        if ((x*x + y*y) < 1.0) {
          k++;
        }
    }
    return 4.0*((double) k)/((double) n);
}

"""

const Clib = tempname()   # ... make a temporary file


In [ ]:
# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_pi(N::Int64) = ccall(("c_pi", Clib), Float64, (Clong,), N)

In [ ]:
randomize();
c_pi(1000000)

In [ ]:
using BenchmarkTools
@benchmark c_pi(1000000)

In [ ]:
# Same can be done in Python

using PyCall

In [ ]:
py"""
import random
def py_pi(n):
  k = 0
  for i in range(n):
    x = random.uniform(0.0,1.0)
    y = random.uniform(0.0,1.0)
    if (x*x + y*y) < 1.0:
      k = k + 1
  return (4.0*k)/n
"""

py_PI = py"py_pi"

In [ ]:
pycall(py_PI,PyAny,1000000)

In [ ]:
@benchmark pycall(py_PI,PyAny,1000000)

In [ ]:
### Reprise the Julia function

normsq(a,b) = a*a + b*b;

function ju_pi(n::Integer)
  @assert n > 0
  k = 0
  for i = 1:n
    if normsq(rand(),rand()) < 1
       k += 1
    end
  end
  4.0 * (k / n)
end

In [ ]:
@benchmark ju_pi(1000000) samples=100

---

# Factorial Fun


In [ ]:
# The 'usual' defintion of factorial function
# A simple loop is much quicker

function fac(n::Integer)
  @assert n > 0
  (n == 1) ? 1 : n*fac(n-1)
end

In [ ]:
dump(:(n*fac(n-1)))

In [ ]:
for i = 1:30
  @printf "%3d : %d \n" i fac(i)
end

In [ ]:
# Passing a big integer returns a BIG integer

fac(big(40))

In [ ]:
gamma(41)     # Γ(n+1)  <=>  n!

In [ ]:
for i = 1:30
  @printf "%3d : %d \n" i fac(big(i))
end

In [ ]:
macroexpand(:(@printf "%3d : %d \n" i fac(big(i))))

In [ ]:
# A non-recursive one liner
#
facA(N::Integer) = N < 1 ? throw(ArgumentError("N must be positive")) : reduce(*,1,collect(big.(1:N)))

@time facA(40)

---

## Fibonacci Series

![](Images/fibimages.png)

![](Images/fibunnies.gif)


In [ ]:
# The 'standard' definition

function fib(k::Integer)
  @assert k > 0
  (k < 3) ? 1 : fib(k-1) + fib(k-2)
end

@time fib(10)

In [ ]:
# But this runs into problems (why?)

@time fib(42)

In [ ]:
# A better version

function fibA(n::Integer)
  @assert n > 0
  a = Array{typeof(n)}(n)
  a[1] = 0
  a[2] = 1
  for i = 3:n
    a[i] = a[i-1] + a[i-2]
  end
  return a[n]
end

![](Images/julia1a.gif)

In [ ]:
@time fibA(big(402))

In [ ]:
# Tail recursive version

fib_tail(a,b,n) = (n > 1) ? fib_tail(b, a+b, n-1) : a
fibB(n) = fib_tail(1, 1, n)

@time fibB(big(402))

In [ ]:
# Non-recursive, no memory usage
# It wiil return a BIG, so no typed instability

function fibC(n::Integer)
  @assert n > 2
  a = b = big(1)
  while n > 1
    (a, b) = (b, a+b)
    n -= 1
  end
  return a
end

@time fibC(402)

# Golden Ratio

Two quantities (a,b, a>b) are in the golden ratio if   a/b = (a+b)/a 

i.e.   ψ = (1 + sqrt(5))/2 ~= 1.618034

Some twentieth-century artists and architects, including Le Corbusier and Dalí, have proportioned their works to approximate the golden ratio—especially in the form of the golden rectangle, in which the ratio of the longer sideto the shorter is the golden ratio—believing this proportion to be aesthetically pleasing. 

Mathematicians since Euclid have studied the properties of the golden ratio, including its appearance in the dimensions of a regular pentagon and in a golden rectangle, which may be cut into a square and a smaller rectanglewith the same aspect ratio. The golden ratio has also been used to analyze the proportions of natural objects as well as man-made systems such as financial markets, in some cases based on dubious fits to data.

![](Images/GR-3.jpg)


In [ ]:
# ψ = (1 + sqrt(5))/2 ~= 1.618034
#
ψ = fibC(402)/fibC(401)

In [ ]:
# This is a Julia built-in constant

golden


---

# The Basel problem

The Basel problem is a problem in mathematical analysis with relevance to number theory, 
first posed by Pietro Mengoli in 1644 and solved by Leonhard Euler in 1734

It asks for the precise summation of the reciprocals of the squares of the natural numbers
<br/><br/>

## $\sum_{i=1}^\infty \frac{1}{i^2} = \frac{\pi^2}{6}$

---


In [ ]:
function basel(N::Integer)
  @assert N > 0
  s = 0.0
  for i = 1:N
    s += 1.0/float(i)^2     
  end 
  return s
end

basel(10000)

In [ ]:
abs(π - sqrt(6.0*basel(10000)))

In [ ]:
@time basel(10^4)

In [ ]:
push!(LOAD_PATH,pwd())

In [ ]:
N = 100_000_000
ccall((:basel, "libmyfuns.dylib"), Cdouble, (Clong,), N)

---

<a href='https://julialang.org/ecosystems' target='_blank()'>
<img src='Images/SS12.png'/>
</a>

<a href='https://juliastats.github.io/' target='_blank()'><img src='Images/SS43.png'></a>
<a href='https://juliaml.github.io/'    target='_blank()'><img src='Images/SS45.png'></a>
<a href='https://github.com/JuliaMath'  target='_blank()'><img src='Images/SS44.png'></a>
<a href='https://github.com/JuliaGPU'   target='_blank()'><img src='Images/SS46.png'></a>


---

![](Images/SS37.png)

# Useful *(but not complete)* packages
## Statistics
- Distributions
- DataFrames
- GLM
- Loess
- TimeSeries
- Klara
- RMath
- Clustering
- Mamba

## Machine Learning &amp; GPUs
- Mocha
- MXNet
- Knet
- Flex
- DecisionTree
- ScikitLearn
- TensorFlow
- NaiveBayes
- BayesNets
<br/><br/>
- CUDA(\*), CU(\*)
- OpenCL, CL(\*)
- GPUBenchmarks


 ---

![](Images/SS90.png)